# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2225257494534,
 'week52high': 146.53,
 'week52low': 57.54,
 'week52change': 0.7430200052210312,
 'sharesOutstanding': 17381624423,
 'float': 0,
 'avg10Volume': 150280885,
 'avg30Volume': 121397155,
 'day200MovingAvg': 119.34,
 'day50MovingAvg': 133.03,
 'employees': 0,
 'ttmEPS': 3.82,
 'ttmDividendRate': 0.8416861162413629,
 'dividendYield': 0.00611748216667923,
 'nextDividendDate': '2021-02-01',
 'exDividendDate': '2020-10-31',
 'nextEarningsDate': '2021-01-15',
 'peRatio': 34.99401141968288,
 'beta': 1.1936189987610941,
 'maxChangePercent': 50.363774435317,
 'year5ChangePercent': 4.85366220734723,
 'year2ChangePercent': 2.2676157010388494,
 'year1ChangePercent': 0.739483076978028,
 'ytdChangePercent': -0.005531980095939273,
 'month6ChangePercent': 0.2574741349776883,
 'month3ChangePercent': 0.2220383628224206,
 'month1ChangePercent': -0.00561646969718829,
 'day30ChangePercent': -0.005643901847218843,
 'day5ChangePercent': -0.052639352901236

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.739483076978028

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
            ignore_index=True
        )
#final_df

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(drop=True, inplace=True)
final_df

Ticker    Price One-Year Price Return Number of Shares to Buy
0    CARR    38.80               2.25643                     N/A
1     FCX    26.92               1.43991                     N/A
2    NVDA   520.59               1.23437                     N/A
3     ALB   167.14               1.09235                     N/A
4    ALGN   541.27                1.0875                     N/A
5    PYPL   237.14                1.0834                     N/A
6     WST   309.82              0.939898                     N/A
7    QCOM   157.66              0.919344                     N/A
8    ABMD   358.93              0.876172                     N/A
9      DE   295.30              0.855739                     N/A
10    AMD    89.47              0.851738                     N/A
11    PWR    70.90              0.846738                     N/A
12   SIVB   457.18              0.828782                     N/A
13   CDNS   136.38              0.818691                     N/A
14    URI   251.86               0.80555                     N/A
15     LB    42.35              0.800891                     N/A
16   IDXX   487.18              0.796066                     N/A
17   IPGP   229.59              0.758178                     N/A
18   SNPS   256.59              0.751171                     N/A
19   AAPL   136.32              0.738989                     N/A
20    CMG  1525.00              0.733313                     N/A
21   KLAC   292.47              0.726912                     N/A
22   EBAY    56.56              0.717103                     N/A
23   AMAT    98.82              0.695025                     N/A
24    TGT   186.81              0.691006                     N/A
25    FDX   241.07              0.683019                     N/A
26   LRCX   494.30              0.655709                     N/A
27    TMO   533.20               0.65372                     N/A
28   ZBRA   389.58              0.652387                     N/A
29   TMUS   129.01              0.636724                     N/A
30   TTWO   205.79              0.627361                     N/A
31    NOW   551.38              0.624704                     N/A
32   AMZN  3344.70              0.619626                     N/A
33   QRVO   174.02              0.618282                     N/A
34     DD    81.22              0.616212                     N/A
35    BIO   578.66              0.613354                     N/A
36   FFIV   201.91              0.609625                     N/A
37    PKI   152.30              0.603452                     N/A
38   ATVI    91.00               0.59262                     N/A
39   APTV   138.60              0.591211                     N/A
40   XLNX   133.29              0.581053                     N/A
41   DXCM   377.73              0.571459                     N/A
42   ALXN   157.74              0.568594                     N/A
43   NFLX   546.45              0.567968                     N/A
44    UPS   160.00              0.560114                     N/A
45    MTD  1201.60               0.55846                     N/A
46   AVGO   465.30               0.55808                     N/A
47   TWTR    52.78              0.557477                     N/A
48   MKTX   554.31              0.553775                     N/A
49     GM    51.24              0.547582                     N/A

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except:
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

In [10]:
portfolio_input()
position_size = float(portfolio_size)/len(final_df.index)
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])

final_df

Enter the size of your portfolio:250000


Ticker    Price One-Year Price Return  Number of Shares to Buy
0    CARR    38.80               2.25643                      128
1     FCX    26.92               1.43991                      185
2    NVDA   520.59               1.23437                        9
3     ALB   167.14               1.09235                       29
4    ALGN   541.27                1.0875                        9
5    PYPL   237.14                1.0834                       21
6     WST   309.82              0.939898                       16
7    QCOM   157.66              0.919344                       31
8    ABMD   358.93              0.876172                       13
9      DE   295.30              0.855739                       16
10    AMD    89.47              0.851738                       55
11    PWR    70.90              0.846738                       70
12   SIVB   457.18              0.828782                       10
13   CDNS   136.38              0.818691                       36
14    URI   251.86               0.80555                       19
15     LB    42.35              0.800891                      118
16   IDXX   487.18              0.796066                       10
17   IPGP   229.59              0.758178                       21
18   SNPS   256.59              0.751171                       19
19   AAPL   136.32              0.738989                       36
20    CMG  1525.00              0.733313                        3
21   KLAC   292.47              0.726912                       17
22   EBAY    56.56              0.717103                       88
23   AMAT    98.82              0.695025                       50
24    TGT   186.81              0.691006                       26
25    FDX   241.07              0.683019                       20
26   LRCX   494.30              0.655709                       10
27    TMO   533.20               0.65372                        9
28   ZBRA   389.58              0.652387                       12
29   TMUS   129.01              0.636724                       38
30   TTWO   205.79              0.627361                       24
31    NOW   551.38              0.624704                        9
32   AMZN  3344.70              0.619626                        1
33   QRVO   174.02              0.618282                       28
34     DD    81.22              0.616212                       61
35    BIO   578.66              0.613354                        8
36   FFIV   201.91              0.609625                       24
37    PKI   152.30              0.603452                       32
38   ATVI    91.00               0.59262                       54
39   APTV   138.60              0.591211                       36
40   XLNX   133.29              0.581053                       37
41   DXCM   377.73              0.571459                       13
42   ALXN   157.74              0.568594                       31
43   NFLX   546.45              0.567968                        9
44    UPS   160.00              0.560114                       31
45    MTD  1201.60               0.55846                        4
46   AVGO   465.30               0.55808                       10
47   TWTR    52.78              0.557477                       94
48   MKTX   554.31              0.553775                        9
49     GM    51.24              0.547582                       97

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [49]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
        pd.Series(
        [
    symbol,
    data[symbol]['price'],
    'N/A',
    data[symbol]['stats']['year1ChangePercent'],
    'N/A',
    data[symbol]['stats']['month6ChangePercent'],
    'N/A',
    data[symbol]['stats']['month3ChangePercent'],
    'N/A',
    data[symbol]['stats']['month1ChangePercent'],
     'N/A',
    'N/A'
        ],
            index = hqm_columns),
            ignore_index = True
        )

In [50]:
hqm_df.fillna(value=0, inplace=True)
hqm_df

Ticker    Price Number of Shares to Buy  One-Year Price Return  \
0        A  126.600                     N/A               0.477416   
1      AAL   17.430                     N/A              -0.365096   
2      AAP  156.900                     N/A               0.147156   
3     AAPL  140.360                     N/A               0.747015   
4     ABBV  105.100                     N/A               0.346531   
5      ABC  110.832                     N/A               0.244189   
6     ABMD  362.310                     N/A               0.898970   
7      ABT  128.600                     N/A               0.442430   
8      ACN  258.040                     N/A               0.199242   
9     ADBE  479.210                     N/A               0.316586   
10     ADI  154.610                     N/A               0.370687   
11     ADM   50.660                     N/A               0.159166   
12     ADP  175.450                     N/A              -0.013609   
13    ADSK  288.920                     N/A               0.416889   
14     AEE   75.930                     N/A              -0.096816   
15     AEP   82.100                     N/A              -0.203196   
16     AES   25.204                     N/A               0.270483   
17     AFL   46.097                     N/A              -0.104464   
18     AIG   38.640                     N/A              -0.225278   
19     AIV    4.780                     N/A              -0.313882   
20     AIZ  133.110                     N/A               0.061396   
21     AJG  116.300                     N/A               0.144918   
22    AKAM  117.080                     N/A               0.196827   
23     ALB  170.680                     N/A               1.093165   
24    ALGN  535.091                     N/A               1.068766   
25     ALK   49.990                     N/A              -0.249594   
26     ALL  112.280                     N/A              -0.079063   
27    ALLE  110.100                     N/A              -0.163947   
28    ALXN  160.850                     N/A               0.544395   
29    AMAT  103.790                     N/A               0.710342   
..     ...      ...                     ...                    ...   
475     VZ   54.627                     N/A              -0.028413   
476    WAB   76.490                     N/A               0.012326   
477    WAT  269.756                     N/A               0.190868   
478    WBA   49.530                     N/A               0.032029   
479    WDC   59.236                     N/A              -0.129114   
480    WEC   90.970                     N/A              -0.088647   
481   WELL   62.275                     N/A              -0.252192   
482    WFC   30.960                     N/A              -0.343704   
483    WHR  184.600                     N/A               0.321419   
484   WLTW  209.630                     N/A              -0.025759   
485     WM  115.430                     N/A              -0.068811   
486    WMB   21.560                     N/A               0.114553   
487    WMT  140.490                     N/A               0.256331   
488    WRB   64.787                     N/A              -0.150974   
489    WRK   41.230                     N/A               0.100452   
490    WST  301.159                     N/A               0.955279   
491     WU   23.120                     N/A              -0.141305   
492     WY   31.228                     N/A               0.099334   
493   WYNN  102.560                     N/A              -0.209836   
494    XEL   64.560                     N/A              -0.052556   
495   XLNX  138.520                     N/A               0.576213   
496    XOM   45.970                     N/A              -0.226965   
497   XRAY   54.390                     N/A              -0.037336   
498    XRX   23.442                     N/A              -0.377252   
499    XYL  100.650                     N/A               0.200343   
500    YUM  10

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [84]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
        ]

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile' 
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(a=hqm_df[change_col].to_list(), score=hqm_df.loc[row, change_col])/100
        
hqm_df

Ticker     Price  Number of Shares to Buy  One-Year Price Return  \
0     ALB   170.680                    11717               1.093165   
1    SIVB   450.740                     4437               0.838638   
2    VIAC    51.320                    38971               0.496542   
3   DISCA    41.970                    47653               0.420950   
4      GM    53.200                    37593               0.562556   
5    AMAT   103.790                    19269               0.710342   
6     FCX    28.896                    69213               1.452232   
7   DISCK    34.870                    57355               0.274413   
8     LLY   215.383                     9285               0.526196   
9    FFIV   198.760                    10062               0.619120   
10    IVZ    21.640                    92421               0.273893   
11     DD    75.160                    26609               0.590957   
12    MOS    27.930                    71607               0.335387   
13   KLAC   289.751                     6902               0.732368   
14   NWSA    20.129                    99359               0.470140   
15     LB    43.200                    46296               0.807180   
16    URI   250.050                     7998               0.821589   
17     MU    82.870                    24134               0.497964   
18     DE   303.690                     6585               0.893581   
19    NWL    24.922                    80250               0.304636   
20    NWS    19.657                   101744               0.385607   
21   APTV   138.680                    14421               0.604133   
22   KEYS   144.000                    13888               0.541570   
23      F    10.883                   183772               0.195442   
24    KSS    44.540                    44903               0.069616   
25   QCOM   163.870                    12204               0.905193   
26    STX    67.790                    29502               0.227204   
27   AVGO   479.320                     4172               0.558734   
28    TPR    32.510                    61519               0.245086   
29   NTAP    69.710                    28690               0.302464   
30   NLSN    23.275                    85929               0.115536   
31   QRVO   182.480                    10960               0.615091   
32   LRCX   510.270                     3919               0.669851   
33   ZBRA   398.000                     5025               0.630280   
34   ABMD   362.310                     5520               0.898970   
35    AES    25.204                    79352               0.270483   
36    CMG  1535.390                     1302               0.740112   
37    TGT   185.864                    10760               0.699429   
38   ILMN   461.220                     4336               0.477882   
39   ANET   316.400                     6321               0.385362   
40    RHI    67.757                    29517               0.189961   
41    COF   103.540                    19316               0.060675   
42    KMX   121.960                    16398               0.224201   
43     GS   273.780                     7305               0.175357   
44   CARR    40.090                    49887               2.249929   
45   ALGN   535.091                     3737               1.068766   
46     RF    17.502                   114272               0.146221   
47     CF    43.650                    45819               0.068986   
48   CTVA    40.490                    49394               0.410767   
49    DVA   122.920                    16270               0.480669   

   One-Year Return Percentile  Six-Month Price Return  \
0                        0.96                1.029880   
1                        0.86                0.960209   
2                         0.5                0.913460   
3                        0.42                0.990052   
4                         0.6                1.061859   
5                        0.76          

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [85]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_df

Ticker     Price  Number of Shares to Buy  One-Year Price Return  \
0     ALB   170.680                    11717               1.093165   
1    SIVB   450.740                     4437               0.838638   
2    VIAC    51.320                    38971               0.496542   
3   DISCA    41.970                    47653               0.420950   
4      GM    53.200                    37593               0.562556   
5    AMAT   103.790                    19269               0.710342   
6     FCX    28.896                    69213               1.452232   
7   DISCK    34.870                    57355               0.274413   
8     LLY   215.383                     9285               0.526196   
9    FFIV   198.760                    10062               0.619120   
10    IVZ    21.640                    92421               0.273893   
11     DD    75.160                    26609               0.590957   
12    MOS    27.930                    71607               0.335387   
13   KLAC   289.751                     6902               0.732368   
14   NWSA    20.129                    99359               0.470140   
15     LB    43.200                    46296               0.807180   
16    URI   250.050                     7998               0.821589   
17     MU    82.870                    24134               0.497964   
18     DE   303.690                     6585               0.893581   
19    NWL    24.922                    80250               0.304636   
20    NWS    19.657                   101744               0.385607   
21   APTV   138.680                    14421               0.604133   
22   KEYS   144.000                    13888               0.541570   
23      F    10.883                   183772               0.195442   
24    KSS    44.540                    44903               0.069616   
25   QCOM   163.870                    12204               0.905193   
26    STX    67.790                    29502               0.227204   
27   AVGO   479.320                     4172               0.558734   
28    TPR    32.510                    61519               0.245086   
29   NTAP    69.710                    28690               0.302464   
30   NLSN    23.275                    85929               0.115536   
31   QRVO   182.480                    10960               0.615091   
32   LRCX   510.270                     3919               0.669851   
33   ZBRA   398.000                     5025               0.630280   
34   ABMD   362.310                     5520               0.898970   
35    AES    25.204                    79352               0.270483   
36    CMG  1535.390                     1302               0.740112   
37    TGT   185.864                    10760               0.699429   
38   ILMN   461.220                     4336               0.477882   
39   ANET   316.400                     6321               0.385362   
40    RHI    67.757                    29517               0.189961   
41    COF   103.540                    19316               0.060675   
42    KMX   121.960                    16398               0.224201   
43     GS   273.780                     7305               0.175357   
44   CARR    40.090                    49887               2.249929   
45   ALGN   535.091                     3737               1.068766   
46     RF    17.502                   114272               0.146221   
47     CF    43.650                    45819               0.068986   
48   CTVA    40.490                    49394               0.410767   
49    DVA   122.920                    16270               0.480669   

   One-Year Return Percentile  Six-Month Price Return  \
0                        0.96                1.029880   
1                        0.86                0.960209   
2                         0.5                0.913460   
3                        0.42                0.990052   
4                         0.6                1.061859   
5                        0.76          

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [86]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop=True, inplace=True)
hqm_df

Ticker     Price  Number of Shares to Buy  One-Year Price Return  \
0     ALB   170.680                    11717               1.093165   
1   DISCA    41.970                    47653               0.420950   
2    SIVB   450.740                     4437               0.838638   
3    VIAC    51.320                    38971               0.496542   
4      GM    53.200                    37593               0.562556   
5     FCX    28.896                    69213               1.452232   
6   DISCK    34.870                    57355               0.274413   
7     IVZ    21.640                    92421               0.273893   
8    AMAT   103.790                    19269               0.710342   
9     KSS    44.540                    44903               0.069616   
10    LLY   215.383                     9285               0.526196   
11   FFIV   198.760                    10062               0.619120   
12    MOS    27.930                    71607               0.335387   
13     LB    43.200                    46296               0.807180   
14     DE   303.690                     6585               0.893581   
15   KLAC   289.751                     6902               0.732368   
16   NWSA    20.129                    99359               0.470140   
17     MU    82.870                    24134               0.497964   
18     DD    75.160                    26609               0.590957   
19    URI   250.050                     7998               0.821589   
20    NWL    24.922                    80250               0.304636   
21   NLSN    23.275                    85929               0.115536   
22   APTV   138.680                    14421               0.604133   
23      F    10.883                   183772               0.195442   
24   ILMN   461.220                     4336               0.477882   
25   ABMD   362.310                     5520               0.898970   
26    NWS    19.657                   101744               0.385607   
27    TPR    32.510                    61519               0.245086   
28   ALGN   535.091                     3737               1.068766   
29   NTAP    69.710                    28690               0.302464   
30   KEYS   144.000                    13888               0.541570   
31   QCOM   163.870                    12204               0.905193   
32    COF   103.540                    19316               0.060675   
33    STX    67.790                    29502               0.227204   
34    KMX   121.960                    16398               0.224201   
35     CF    43.650                    45819               0.068986   
36   ANET   316.400                     6321               0.385362   
37   LRCX   510.270                     3919               0.669851   
38    CMG  1535.390                     1302               0.740112   
39   CARR    40.090                    49887               2.249929   
40   AVGO   479.320                     4172               0.558734   
41    AES    25.204                    79352               0.270483   
42     RF    17.502                   114272               0.146221   
43   ZBRA   398.000                     5025               0.630280   
44    TGT   185.864                    10760               0.699429   
45   QRVO   182.480                    10960               0.615091   
46     GS   273.780                     7305               0.175357   
47    RHI    67.757                    29517               0.189961   
48   CTVA    40.490                    49394               0.410767   
49    DVA   122.920                    16270               0.480669   

   One-Year Return Percentile  Six-Month Price Return  \
0                        0.96                1.029880   
1                        0.42                0.990052   
2                        0.86                0.960209   
3                         0.5                0.913460   
4                         0.6                1.061859   
5                        0.98          

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [89]:
portfolio_input()

Enter the size of your portfolio:1000000


In [90]:
position_size = float(portfolio_size)/len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])

hqm_df

Ticker     Price  Number of Shares to Buy  One-Year Price Return  \
0     ALB   170.680                      117               1.093165   
1   DISCA    41.970                      476               0.420950   
2    SIVB   450.740                       44               0.838638   
3    VIAC    51.320                      389               0.496542   
4      GM    53.200                      375               0.562556   
5     FCX    28.896                      692               1.452232   
6   DISCK    34.870                      573               0.274413   
7     IVZ    21.640                      924               0.273893   
8    AMAT   103.790                      192               0.710342   
9     KSS    44.540                      449               0.069616   
10    LLY   215.383                       92               0.526196   
11   FFIV   198.760                      100               0.619120   
12    MOS    27.930                      716               0.335387   
13     LB    43.200                      462               0.807180   
14     DE   303.690                       65               0.893581   
15   KLAC   289.751                       69               0.732368   
16   NWSA    20.129                      993               0.470140   
17     MU    82.870                      241               0.497964   
18     DD    75.160                      266               0.590957   
19    URI   250.050                       79               0.821589   
20    NWL    24.922                      802               0.304636   
21   NLSN    23.275                      859               0.115536   
22   APTV   138.680                      144               0.604133   
23      F    10.883                     1837               0.195442   
24   ILMN   461.220                       43               0.477882   
25   ABMD   362.310                       55               0.898970   
26    NWS    19.657                     1017               0.385607   
27    TPR    32.510                      615               0.245086   
28   ALGN   535.091                       37               1.068766   
29   NTAP    69.710                      286               0.302464   
30   KEYS   144.000                      138               0.541570   
31   QCOM   163.870                      122               0.905193   
32    COF   103.540                      193               0.060675   
33    STX    67.790                      295               0.227204   
34    KMX   121.960                      163               0.224201   
35     CF    43.650                      458               0.068986   
36   ANET   316.400                       63               0.385362   
37   LRCX   510.270                       39               0.669851   
38    CMG  1535.390                       13               0.740112   
39   CARR    40.090                      498               2.249929   
40   AVGO   479.320                       41               0.558734   
41    AES    25.204                      793               0.270483   
42     RF    17.502                     1142               0.146221   
43   ZBRA   398.000                       50               0.630280   
44    TGT   185.864                      107               0.699429   
45   QRVO   182.480                      109               0.615091   
46     GS   273.780                       73               0.175357   
47    RHI    67.757                      295               0.189961   
48   CTVA    40.490                      493               0.410767   
49    DVA   122.920                      162               0.480669   

   One-Year Return Percentile  Six-Month Price Return  \
0                        0.96                1.029880   
1                        0.42                0.990052   
2                        0.86                0.960209   
3                         0.5                0.913460   
4                         0.6                1.061859   
5                        0.98          

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [91]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name= 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [92]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [93]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [94]:
writer.save()